In [1]:
import pandas as pd

# Importando tabelas

In [2]:
df_dados = pd.read_excel('dados.xlsx')
df_fator = pd.read_excel('fatores.xlsx')

lst_meses = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
lst_meses_dados = ['Dez anterior','Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov']

# Gerando Data Frame das diferenças

In [3]:
df_diferença = df_dados[['Galpões', 'Complemento', 'Empresa', 'Tipo de imóvel']]

for m, d in zip(lst_meses, lst_meses_dados):
    dif = df_dados.loc[:, m] - df_dados.loc[:, d]
    df_diferença.loc[:, m] = dif
display(df_fator)

C:\Users\Gabriel\AppData\Local\Temp\ipykernel_7432\4104899759.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_diferença.loc[:, m] = dif
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_7432\4104899759.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_diferença.loc[:, m] = dif
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_7432\4104899759.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

,Volume min de água m3,Volume max de água m3,Tipo de fator,Fator Industrial,Fator Residencial
0,0,10,Fixo,51.04,25.42
1,10,20,Multiplicador,6.05,3.54
2,20,50,Multiplicador,9.76,5.45
3,50,9999999999,Multiplicador,11.46,6.51


# Gerando Data Frame dos preços

In [4]:
# lst_cabeçalho = ['Galpões', 'Complemento','Registro', 'Empresa', 'Tipo de imovel']
# df_preços = df_dados[lst_cabeçalho]

def df_taxas_xm(index, m, type):
    df_temp = df_preço.loc[(df_diferença[m] < df_fator.loc[index,'Volume max de água m3']) & (df_diferença['Tipo de imóvel'] == type), m]
    return df_temp

def df_taxas_Mxm(index, m, type):
    #type foi utilizada fora da função como o tipo de imóvel. No caso do loop, como var
    df_temp = df_preço.loc[(df_diferença[m] < df_fator.loc[index,'Volume max de água m3']) & (df_diferença['Tipo de imóvel'] == type) & (df_diferença[m] >= df_fator.loc[index, 'Volume min de água m3']), m]
    return df_temp

def df_taxas_Mx(index, m, type):
    df_temp = df_preço.loc[(df_diferença[m] >= df_fator.loc[index,'Volume min de água m3']) & (df_diferença['Tipo de imóvel'] == type), m]
    return df_temp


df_preço = df_diferença

for m in lst_meses:
    
    for type in ['Industrial', 'Residencial']:
        
        df_preço.loc[(df_diferença[m] < df_fator.loc[0,'Volume max de água m3']) & (df_diferença['Tipo de imóvel'] == type), m] = df_fator.loc[0, f'Fator {type}']
        
        for index in range(1, 4):
            df_temp = df_diferença.loc[(df_diferença[m] < df_fator.loc[index,'Volume max de água m3']) & (df_diferença['Tipo de imóvel'] == type) & (df_diferença[m] >= df_fator.loc[index, 'Volume min de água m3']), m]
            df_preço.loc[(df_diferença[m] < df_fator.loc[index,'Volume max de água m3']) & (df_diferença['Tipo de imóvel'] == type) & (df_diferença[m] >= df_fator.loc[index, 'Volume min de água m3']), m] = df_temp * df_fator.loc[index, f'Fator {type}']
            
        # df_temp = df_diferença.loc[(df_diferença[m] >= df_fator.loc[3,'Volume min de água m3']) & (df_diferença['Tipo de imóvel'] == type), m]
        # df_preço.loc[(df_diferença[m] >= df_fator.loc[3,'Volume min de água m3']) & (df_diferença['Tipo de imóvel'] == type), m] = df_temp * df_fator.loc[3, f'Fator {type}']











# display(df_preço)
# df_preço.loc[df_diferença.loc[:, 'Jan':'Fev'] < 10, 'Jan':'Fev'] = 10
# display(df_preço)
# df_preço.loc[df_diferença.loc[:, lst_meses] < df_fator.loc[0, 'Volume max de água m3'], lst_meses] = df_fator.loc[0, 'Fator Industrial']








# display(df_diferença.merge(df))



# def taxa(index):
#     # index\: index do df_fator
#     # dataframe\: dataframe base
#     # type\: tipo de imóvel
    
#     df = df_diferença
    
#     lst_cabeçalho = df.columns

#     fixo_industrial = df_fator.loc[0,'Fator Industrial']
#     fixo_residencial = df_fator.loc[0,'Fator Residencial']

#     if df_fator.loc[index, 'Tipo de fator'] == 'Fixo':
        
#         df_temp_1 = df.loc[:,].merge(df.loc[df[[lst_meses]] < df_fator.loc[index, 'Volume max de água m3'] and df['Tipo de imóvel'] == 'Industrial', lst_meses]*0 + fixo_industrial)
#         df_temp_2 = df.loc[df[[lst_meses]] < df_fator.loc[index, 'Volume max de água m3'] and df['Tipo de imóvel'] == 'Residencial', lst_meses]*0 + fixo_residencial
#         return df_temp_1.merge(df_temp_2)
    
#     elif df_fator.loc[index, 'Tipo de fator'] == 'Multiplicador':

#         df_temp_1 = df.loc[, lst_cabeçalho] * 









# df_dados.merge()
# fixo_industrial = df_fator.loc[0,'Fator Industrial']
# fixo_residencial = df_fator.loc[0,'Fator Residencial']


# lst_cabeçalho = df_diferença.columns
# df_temp_ind_fixo = df_diferença.loc[df_diferença[lst_meses] < df_fator.loc[0, 'Volume max de água m3'] and df_diferença['Tipo de imóvel'] == 'Industrial', lst_cabeçalho]*0 + fixo_industrial
# df_temp_res_fixo = df_diferença.loc[df_diferença[m] < df_fator.loc[0, 'Volume max de água m3'] and df_diferença['Tipo de imóvel'] == 'Residencial', lst_cabeçalho]*0 + fixo_residencial
# df_temp_ind_1



# for m in lst_meses:
#     if df_diferença[m] < df_fator.loc[0, 'Volume max de água m3'] and df_diferença['Tipo de imovel'] == 'Industrial':
#         df_preços[m] = fixo_industrial
#     elif df_diferença[m] < df_fator.loc[0, 'Volume max de água m3'] and df_diferença['Tipo de imovel'] == 'Residencial':
#         df_preços[m] = fixo_residencial
#     elif df_diferença[m] >= df_fator.loc[1, 'Volume min de água m3'] and df_diferença[m] < df_fator.loc[1, 'Volume max de água m3'] and df_diferença['Tipo de imovel'] == 'Industrial':
#         df_preços[m] = 


0    105.8
Name: Jan, dtype: float64

,Galpões,Complemento,Empresa,Tipo de imóvel,Jan,Fev,Mar,Abr,Mai,Jun,Jul,Ago,Set,Out,Nov,Dez
0,Galpão 1,NaN,Abaixo de Zero,Industrial,105.8000,67.50000,74.80000,53.80000,75.00000,68.6000,NaN,NaN,NaN,NaN,NaN,NaN
1,Galpão 2,NaN,Masp,Industrial,370.0016,361.51040,437.44320,420.36320,426.60960,467.2112,NaN,NaN,NaN,NaN,NaN,NaN
2,Galpão 3,NaN,NaN,Industrial,51.0400,51.04000,51.04000,51.04000,51.04000,51.0400,NaN,NaN,NaN,NaN,NaN,NaN
3,Galpão 4,NaN,Metal Printing,Industrial,413.4336,460.28160,406.11360,57.60000,54.95000,419.5824,NaN,NaN,NaN,NaN,NaN,NaN
4,Marcenaria/Serralheria,NaN,NaN,Industrial,51.0400,51.04000,51.04000,51.04000,51.04000,51.0400,NaN,NaN,NaN,NaN,NaN,NaN
5,Galpão 5,Reservatório,NaN,Industrial,51.0400,51.04000,93.71450,51.04000,51.04000,51.0400,NaN,NaN,NaN,NaN,NaN,NaN
6,Galpão 5,Piso Superior e WC térreo,NaN,Industrial,51.0400,51.04000,51.04000,51.04000,51.04000,51.0400,NaN,NaN,NaN,NaN,NaN,NaN
7,Galpão 6,Piso Térreo,NaN,Industrial,51.0400,51.04000,51.04000,51.04000,51.04000,51.0400,NaN,NaN,NaN,NaN,NaN,NaN
8,Galpão 6,Piso Intermediário,NaN,Industrial,51.0400,51.04000,51.04000,51.04000,51.04000,51.0400,NaN,NaN,NaN,NaN,NaN,NaN
9,Galpão 6,Piso Superior,NaN,Industrial,51.0400,51.04000,51.04000,51.04000,51.04000,51.0400,NaN,NaN,NaN,NaN,NaN,NaN
